# info
- by: LeeHaEun
- start: 21.11.08 Mon
- end: 21.11.08 Mon
- review: 21.11.08 Mon

# theme: MNIST

- **Flattening**: The process of converting a 2d image into 1d vector
- **ReLU**: 모델이 `비선형 문제`를 해결할 수 있는 활성화 함수
- **Softmax**: 가능한 각 출력 클래스에 대한 `확률`을 제공하는 함수
- **Metrics**:Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified. -> 로스펑션 & 옾티마이저랑 같이 세팅
- **Classification**: A machine learning model used for distinguishing among two or more output **categories**
- **Training Set**: The data used for training the neural network.
- **Test set**: The data used for testing the final performance of our neural network.
- **Regression**: A model that outputs a single value. For example, an estimate of a house’s value.
  - Remember, we use **Softmax** as the activation function in our last Dense layer to create this probability distribution.

# code_l03c01_classifying_images_of_clothing

## 기본 세팅

In [ ]:
!pip install -U tensorflow_datasets

: 텐서플로우 데이터세트 다운로드

In [ ]:
import tensorflow as tf

In [ ]:
# Import TensorFlow Datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

### MNIST dataset import

In [ ]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

* The model is trained using `train_dataset`.
* The model is tested against `test_dataset`.

In [ ]:
class_names = metadata.features['label'].names
print("Class names: {}".format(class_names))

- **label**로 아웃풋 분류

In [ ]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

- 트레이닝 세트랑 테스트 세트 구분

## 데이터 전처리

In [ ]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32) #이미지를 float으로 캐스팅
  images /= 255 #255로 나누기
  return images, labels

# dataset를 normalize해서 덮어씌우기
train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)

# 처음 dataset 사용할 때에는 disk에 로드된다 -> caching하면 memory로 불러온다 -> faster!
train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()

- **tf.cast** = 새로운 형태로 cast
- **map**: 함수를 데이터에 적용
  - → map(함수, 데이터)
  - → 데이터.map(함수)
- **cache()**는 그냥 데이터 저장소인듯

### 전처리된 데이터를 살펴봅시당

In [ ]:
# Take a single image, and remove the color dimension by reshaping
for image, label in test_dataset.take(1): #testset에서 1개만 불러오기->image,label
  break
image = image.numpy().reshape((28,28)) #image를 넘파이 파입으로 저장

# Plot the image - voila a piece of fashion clothing
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

take(): 해당 배치(배치작업은, 데이터를 실시간으로 처리하는게 아니라, 일괄적으로 모아서 처리하는 작업을 의미한다.)를 몇 번 불러올지 정한다


cmap은 색깔 차트

plt는 위에서 matplotlib으로 선언했음

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(train_dataset.take(25)):
    image = image.numpy().reshape((28,28))
    plt.subplot(1,25,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.xlabel(class_names[label])
plt.show()

matplotlib.pyplot 모듈의 subplot(rows, cols, index) 함수는 여러 개의 그래프를 하나의 그림에 나타내도록 합니다.


## 모델 만들기

### 레이어 세팅

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

- "input" `tf.keras.layers.Flatten` = 이미지를 28 × 28 픽셀의 2d 배열에서 784 픽셀(28*28)의 1d 배열로 변환합니다. 이 레이어는 이미지에서 픽셀 행을 분리하여 정렬하는 것으로 간주합니다. 이 계층은 데이터만 재구성하므로 학습할 매개 변수가 없습니다.

- "hidden" `tf.keras.layers.Dense`=128개의 뉴런이 촘촘히 연결된 층입니다. 각 뉴런(또는 노드)은 이전 레이어의 784개 노드 모두에서 입력을 받아 훈련 중에 학습될 숨겨진 파라미터에 따라 입력에 가중치를 부여하고 다음 레이어에 단일 값을 출력한다. `비선형`

- "output" `tf.keras.layers.Dense`=128-뉴론, 그 다음 10-노드 **소프트맥스** 레이어. 각 점은 의류 부류를 나타냅니다. 이전 계층에서와 같이, 최종 계층은 계층 내의 128개 노드로부터 입력을 받아 [0, 1] 범위의 값을 출력하여 이미지가 해당 클래스에 속할 `확률`을 나타냅니다. 10개 노드 값의 합은 1입니다.

모델 컴파일 전에, 분석을 위한 세팅 3단계
1. 옵티마이저
2. 로스 펑션
3. 메트릭스

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

## 모델 학습시키기

먼저, 트레이닝 세트에 대해 정의한다
- **dataset. ... .repeat()**: epoch 만큼 반복해서 학습
- **dataset. ... .shuffle(num_train_examples)**: 모델이 예시에서 아무것도 학습할 수 없도록 순서를 랜덤하게
- **dataset. ... .batch(BATCH_SIZE)**: model.fit이 업데이트할 때 32개 세트를 사용한다는 것을 의미

In [ ]:
BATCH_SIZE = 32
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

In [ ]:
model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

이거 실행시키면 로스 펑션도 같이 나타난다 -> 88%정도 나온다

## 모델 정확도 판단

트레이닝 세트를 통해 모델을 학습시켰으면,

이후 테스트 세트의 예제를 사용해서 정확도를 평가한다!

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

트레이닝 세트의 정확도 > 테스트 세트의 정확도

=> 당연하다. 테스트세트는 학습한적이 없으니까



## 실제로 적용해봅시다

In [ ]:
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [ ]:
predictions.shape #(32,10)
predictions[0] #0번에 관한 예측 확률 제시
np.argmax(predictions[0]) #위의 확률중에 제일 큰거 return
test_labels[0] #실제로 4인것을 확인할 수 있다

In [ ]:
def plot_image(i, predictions_array, true_labels, images):#이미지와 주석
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img[...,0], cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label): ##막대그래프
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
  
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# 예시 하나만 띄워봅시다
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
# 예시 하나만 띄워봅시다
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
# 예시 여러개 한번에 띄워봅시다
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)


In [ ]:
# Grab an image from the test dataset
img = test_images[0]

print(img.shape)

In [ ]:
# Add the image to a batch where it's the only member.
img = np.array([img])

print(img.shape)

In [ ]:
# 실제로 예측해봅시다
predictions_single = model.predict(img)
print(predictions_single)

plot_value_array(0, predictions_single, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

np.argmax(predictions_single[0])